This section will cover the following 

1. Time Series Data Representations 
2. Converting Time Series Object 
3. Making Decomposition Plots 
4. Checking for Stationarity 
5. Applying Rolling Averages


## Load Packages 


In [ ]:
library(crimedata) # package containing the crime data 
library(dplyr) #for data manipulation 
library(fable) # for time series forecasting models
library(tsibble) #for data manipulation 
library(ggplot2) #for plotting
library(lubridate) #for data manipulation 
library(forecast) # to create S/ARIMA models
library(TTR) # to create rolling averages
library(tseries) #to create time series plot 
library(readr) #to read in data
library(ggfortify) #plotting tool for time series/PCA/cluster and survival analysis


## Time Series Data Representation 


We will first start this code demo by exploring some different types of time series data 

Typically when working with data in R, you need to decide the object class of your data at hand. This is important because the object class you choice affects more than the data is stored; it will dictate which functions will be available for data pre-processing, analysing and plotting. 

Typically data in R is stored as a vector. However, when working with time series data we tend to have to convert to the right object class known as a 'time series object' 

R has at least eight different implementations of data structures for representing time series data. The list below identifies the most frequently used packages

> ts #base package 
> zoo 
> xts 
> tsibble 




## King - (non-seasonal)

We will first use a data set named 'Kings' which is an example of a small time series data. It records the age of death for 42 successive king of Enlgand.

The data set can be found here: <https://robjhyndman.com/tsdldata/misc/kings.dat>

AS this data is a text file we can read in the data using the *scan()* function. 


In [ ]:
king <- scan('https://robjhyndman.com/tsdldata/misc/kings.dat', skip = 3) 
king

class(king)


To convert a dataframe to a time series object we can use the 'ts' function from the base package 



In [ ]:
king <- ts(king)

class(king)
king


## Births - Seasonal Data 

What happens if we have data that has been collected at more regular intervals i.e. monthly, weekly, quarterly etc. 

If this is the case you would have to specify the number of times that day was collected pet year using the *frequency* parameter. 

Lets explore this with a different dataset, named 'Births' which refers to the number of births per month in New York City, from 1946 to 1958. 

The dataset can be found here: <"http://robjhyndman.com/tsdldata/data/nybirths.dat">, This time we can call on some more parameters, the code to do so would look like this *ts(vector, start=, end=, frequency=)*


In [ ]:
births <- scan("http://robjhyndman.com/tsdldata/data/nybirths.dat")
births


births <- ts(births, frequency = 12, start = c(1946, 1))
births


## Plotting a Time Series Dataset 

To plot both datasets (seasonal or non-seasonal) you can use the
*plot.ts* function from the ts package. 


In [ ]:
plot.ts(king)
plot.ts(births)


## Plotting TS objects using Ggplot



In [ ]:
library(ggfortify)
autoplot(births)


autoplot(births, ts.geom = "ribbon", fill = "grey")


## Plotting with the Forecast package 

S/ARIMA = 
Seasonal (S) / Autoregressive (AR) / Integrated (I) / Moving Average (MA)


In [ ]:
library(forecast)

#run the model 
births_arima <- auto.arima(births, seasonal = T)
print(summary(births_arima))
checkresiduals(births_arima)

#make the forecast 
births_forecast <- forecast(births_arima, h = 12)
print(births_forecast)

#plot 
autoplot(births_forecast)


## Decomposition

*ts = seasonality (s) + trend (t) + noise (n) + cycle (c)

Additive compononent = [s+t+n*c]
Multiplicative Component = [s*t*n*c]


In [ ]:
birth_decomp <- decompose(births)
head(birth_decomp)
plot(birth_decomp)

# you can improve the graphics by using the autoplot function from the forecast package
births %>% decompose %>% 
  autoplot()


## How to remove seasonality from your data (seasonal adjustment)?

*To seasonally adjust a additive model (Time Series - Seasonal)
*To seasonally adjust a multiplicative model (Time Series / Seasonal)


In [ ]:
adjusted_births <- births - birth_decomp$seasonal
plot(adjusted_births)


## Checking for Stationarity 

To reiterate,a time series is said weakly stationary if 

-  its mean is constant
-  its standard deviation is constant 
-  its cross covariance does not depend on time but depends only on the lag between the two series concerned. 

There are two ways to check for this; 


1. the graphical way 


In [ ]:
ts.plot(king)



2. the statistical method

We can use the Augmented Dickey–Fuller (ADF) t-statistic test to find if the series has a unit root (a series with a trend line will have a unit root and result in a large p-value).


In [ ]:
library(tseries)
adf.test(king)


## The Zoo Package -

The zoo package consists of the methods for totally ordered indexed observations. It aims at performing calculations containing irregular time series of numeric vectors, matrices & factors. The zoo package interfaces to all other time series packages on CRAN.

We are going to look how how to run rolling averages, in order to smooth our data 


In [ ]:
install.packages("zoo")
require("zoo")

#load the average temperature by month and year in Nottingham via the 'nottem' dataset. This exists in base R. 
data("nottem")

#plot 
plot(nottem)

#add some labels via the autoplot functio
theme_set(theme_bw())
autoplot(nottem) + xlab("") + ylab("") + ggtitle("Time Series Plot of the `nottem' Time-Series") + 
    theme(plot.title = element_text(hjust = 0.5))

#lets also view the seasonal subseries as well
ggsubseriesplot(nottem)

#before smoothing lets explore some descriptive statistics
library(xtable)
nottem2 <- xtable(nottem)
head(nottem)
summary(nottem2)

#is the data monthly quartely, yearly etc>
frequency(nottem)


#lets decompose this data and plot the seperate trends
decompose(nottem, type = "additive") 
plot(decompose(nottem, type = "additive"))
#with autoplot()
autoplot(decompose(nottem, type = "additive")) + xlab("") + ylab("") + 
    ggtitle("Decomposition plot of the `nottem' Time-Series") + 
    theme(plot.title = element_text(hjust = 0.5))


#and what about removing the seasonally adjusted data
x <- decompose(nottem, "additive")
nottem_adjusted <- nottem - x$seasonal
plot(nottem_adjusted)
#remember you can plot the separate components like so
plot(x$seasonal)


## What happens when we smooth data?



In [ ]:
#smooth 
not_mean <- rollmean(nottem, 12,fill = NA, align = "right")
not_mean
plot(not_mean)


## The XTS package 

XTS stands for eXtensible time series. It is an extension of the zoo object that includes a matrix and index. Xts is a matrix with associated times for each observation

Lets have a look at how we would convert the nottem time series in a xts object and why we might want to do this.


In [ ]:
library(xts)



#this is how you would convert a df to a xts object 
data("nottem")
xts2 <- as.xts(nottem)
xts2_m <- as.matrix(xts2)
head(xts2_m)


In order to understand the xts objects, let us create an xts object work that stores the number of hours that someone has worked along with some attributes about them like birth date. 



In [ ]:
library(xts)

# Create the object data using 5 random numbers
hours <- rnorm(20, mean = 8)

# Create dates as a Date class object starting from 1922-01-01
dates <- seq(as.Date("1922-01-01"), length = 20, by = "days")

# Create birthday a POSIXct date class object
dob <- as.POSIXct("1900-01-08")

# Create xts object work
work <- xts(x = hours, order.by = dates, born = dob)

#We can look at the structure of an xts object using the str() function.
str(work)


When working with time series, it will sometimes be necessary to separate your time series into its core data and index attributes for additional analysis and manipulation. The core data is the matrix portion of xts. You can separate this from the xts object using coredata(). The index portion of the xts object is available using the index() function. 

## Lets explore the xts object


In [ ]:
# Extract the core data 
hours <- coredata(work)

# View the class of core data
class(hours)

# Extract the index data
index <- index(work)

# View the class of index
class(index)


## Subset the data 

It is often required when working with time series data to filter observations over a certain time range like a week, day or month. 


In [ ]:
last(work, n = '2 weeks')
first(work, n = '2 weeks')
